In [1]:
json_objects = {
  "nodes": {
    "__START__": {
      "schema_info": "",
      "input_schema": "",
      "output_schema": "",
      "description": "Entry point of the graph.",
      "function_name": ""
    },
    "planner": {
      "schema_info": "PlanExecute: TypedDict with fields input (str), plan (List[str]), past_steps (Annotated[List[Tuple], operator.add]), response (str)",
      "input_schema": "PlanExecute",
      "output_schema": "PlanExecute",
      "description": "Plan step generates a plan based on the input using llm structured output functionality, stores it to the plan field",
      "function_name": "plan_step"
    },
    "agent": {
      "schema_info": "PlanExecute: TypedDict with fields input (str), plan (List[str]), past_steps (Annotated[List[Tuple], operator.add]), response (str)",
      "input_schema": "PlanExecute",
      "output_schema": "PlanExecute",
      "description": "Uses llm with tool binding for the stock related queries",
      "function_name": "execute_step"
    },
    "replan": {
      "schema_info": "PlanExecute: TypedDict with fields input (str), plan (List[str]), past_steps (Annotated[List[Tuple], operator.add]), response (str)",
      "input_schema": "PlanExecute",
      "output_schema": "Union[Response, Plan]",
      "description": "Evaluates progress and uses an LLM to either revise the plan or generate a final response.",
      "function_name": "replan_step"
    },
    "__END__": {
      "schema_info": "",
      "input_schema": "",
      "output_schema": "",
      "description": "End point of the graph.",
      "function_name": ""
    }
  },
  "edges": {
    "edge_1": {
      "source": "__START__",
      "target": "planner",
      "routing_conditions": "Start the planning process.",
      "conditional": False
    },
    "edge_2": {
      "source": "planner",
      "target": "agent",
      "routing_conditions": "After planning, execute the first step.",
      "conditional": False
    },
    "edge_3": {
      "source": "agent",
      "target": "replan",
      "routing_conditions": "After executing a step, check if replanning is needed.",
      "conditional": False
    },
    "edge_4": {
      "source": "replan",
      "target": "agent",
      "routing_conditions": "If no response is generated, continue to agent for further execution.",
      "conditional": True
    },
    "edge_5": {
      "source": "replan",
      "target": "__END__",
      "routing_conditions": "If a response is generated, end the process.",
      "conditional": True
    }
  }
}

In [2]:
import import_ipynb
from phase1_edge_handling import edge_builder_agent
from phase1_node_to_code import node_to_code_app

/Users/kanishkgupta/Documents/GitHub/AgentAgent/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
skipList = ["__START__", "__END__"]

In [4]:
import operator
from typing import Annotated, List
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.types import Send
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
import uuid


class NodeEvaluationReport(BaseModel):
    node_name: str
    node_code_stub: str
    edge_code: str

class GraphCompilerState(MessagesState):
    json_objects: dict
    node_reports: Annotated[List[NodeEvaluationReport], operator.add]


class NodeProcessState():
    node_name: str
    node_info: dict
    edge_info: dict

def graph_map_step(state: GraphCompilerState):
      # Get edges originating from the current node
    json_objects = state["json_objects"]
    edges = json_objects["edges"]
    nodes = json_objects["nodes"]
    sends = []
    for node_key, node_val in nodes.items():
        outgoing_edges = list(filter(lambda edge: edge["source"] == node_key, edges.values()))
        sends.append(Send("node_process", {"node_name": node_key, "node_info": node_val, "edge_info": outgoing_edges}))
    return sends

edge_info_prompt = ChatPromptTemplate.from_template("""
<GraphNodeImplementation>
{node_code}
</GraphNodeImplementation>
<EdgeInformation>
{edge_json}
</Edgeinformation>""")

def node_process(state: NodeProcessState):
    uuid_str = uuid.uuid4()
    config = {"configurable": {"thread_id": str(uuid_str)}}
    if state["node_name"] not in skipList:
        for output in node_to_code_app.stream(state["node_info"], config, stream_mode="updates"):
            print(output)
        code= node_to_code_app.get_state(config).values["final_code"]
    else:
        code = "no implementation needed"
    edge_code = edge_builder_agent.invoke({"messages": [HumanMessage(content = edge_info_prompt.format(node_code=code,edge_json=state["edge_info"]))]},config)
    return {
        "node_reports" : [NodeEvaluationReport(node_name=state["node_name"], node_code_stub=code, edge_code=edge_code["messages"][-1].content)] 
    }

code_compiler_prompt = ChatPromptTemplate.from_template("""
You are responsible for merging and compiling the code  for the given graph
<Graph>
{graphEdgeDict}
</Graph>

<GraphNodeImplementation>
{graphImplementations}
</GraphNodeImplementation>

<EdgeImplementation>
{edgeImplementations}
</EdgeImplementation>
""")

def graph_compile(state: GraphCompilerState):
    node_evals : List[NodeEvaluationReport]= state["node_reports"]

    code_stubs = [node_eval.node_code_stub for node_eval in node_evals]
    edge_stubs = [node_eval.edge_code for node_eval in node_evals]
    json_objects = state["json_objects"]
    edges = json_objects["edges"]
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
    response = llm.invoke([SystemMessage(content= code_compiler_prompt.format(
        graphEdgeDict=edges,
        graphImplementations=code_stubs,
        edgeImplementations=edge_stubs))])
    return {
        "messages": [response]
    }


workflow = StateGraph(GraphCompilerState)
workflow.add_node(graph_map_step, "graph_map")
workflow.add_node(node_process, "node_process")
workflow.add_node(graph_compile,"graph_compile")

workflow.add_conditional_edges(START, graph_map_step, ["node_process"])
workflow.add_edge("node_process", "graph_compile")
workflow.add_edge("graph_compile", END)

compiler_graph = workflow.compile()

In [5]:
import uuid
uuid_str1= uuid.uuid4()
config = {"configurable": {"thread_id": str(uuid_str1)}}

for output in compiler_graph.stream({"json_objects": json_objects}, config, stream_mode="updates"
    ):
        print(output)   

{'identify_node': {'node_type': 'planner', 'messages': [HumanMessage(content='\nYou are provided with the following information about the node:\n<SchemaInfo>\nPlanExecute: TypedDict with fields input (str), plan (List[str]), past_steps (Annotated[List[Tuple], operator.add]), response (str)\n</SchemaInfo>\n<InputSchema>\nPlanExecute\n</InputSchema>\n<OutputSchema>\nPlanExecute\n</OutputSchema>\n<Description>\nUses llm with tool binding for the stock related queries\n</Description>\n<FunctionName>\nexecute_step\n</FunctionName>\n\nBelow is the skeleton of the function that you need to implement:\ndef execute_step(state:PlanExecute) -> PlanExecute:\n    """Uses llm with tool binding for the stock related queries"""\n    # Implement the function to meet the description.\n\nthe state is of type PlanExecute and the function is of type PlanExecute\nThe general idea is that the implementation would involve extracting the input from the state, and updating the state with the output. Description

In [5]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

result = llm.invoke("write langgraph code for plan-and-execute, use structured output functionality")

In [6]:
result.pretty_print()

================================== Ai Message ==================================

Certainly! Below is an example of how you can write LangGraph code for a **plan-and-execute** pattern using the **structured output** functionality. This example demonstrates a simple agent that first **plans** a sequence of steps to achieve a goal, then **executes** each step in order.

```langgraph
agent PlanAndExecuteAgent {
  input: string goal

  // Step 1: Plan - generate a structured plan as a list of steps
  step plan {
    prompt: """
    You are an AI assistant. Given the goal below, create a step-by-step plan to achieve it.
    Output the plan as a JSON array of steps, each with a "step_number" and "instruction".

    Goal: {goal}

    Example output:
    [
      {"step_number": 1, "instruction": "Do this first"},
      {"step_number": 2, "instruction": "Then do this"}
    ]

    Your output:
    """
    output: json array of {
      step_number: int,
      instruction: string
    }
  }

  // S

In [3]:
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv
load_dotenv()
model = ChatAnthropic(model='claude-3-7-sonnet-20250219')
result = model.invoke("write langgraph code for plan-and-execute, use structured output functionality")
result.pretty_print()

================================== Ai Message ==================================

# LangGraph Code for Plan-and-Execute with Structured Output

Below is a complete implementation of a plan-and-execute pattern using LangGraph with structured output functionality. This implementation creates a graph that:

1. Takes a user query
2. Plans steps to address the query
3. Executes each step in sequence
4. Provides a final answer

```python
import os
from typing import List, Tuple, TypedDict, Annotated, Literal
from datetime import datetime

from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph import StateGraph, END

# Define structured output schemas
class PlanStep(BaseModel):
    step_number: int = Field(description="The number of this step in the plan")
    step_description: str = Field(description="De

In [9]:
result.pretty_print()

================================== Ai Message ==================================

Certainly! Below is an example of how you might write LangGraph code to implement a **Plan-and-Execute** pattern. This pattern typically involves two main steps:

1. **Plan**: Generate a plan or a sequence of steps to solve a problem.
2. **Execute**: Carry out the plan step-by-step, possibly with feedback or verification after each step.

---

### LangGraph Code for Plan-and-Execute

```langgraph
graph PlanAndExecute {
  
  node Input {
    type: "text"
    description: "User input or problem statement"
  }

  node Planner {
    type: "llm"
    description: "Generate a plan based on the input"
    model: "gpt-4"
    prompt: """
    You are a planner. Given the following problem, generate a step-by-step plan to solve it.

    Problem:
    {Input}

    Plan:
    1.
    """
  }

  node Executor {
    type: "llm"
    description: "Execute each step of the plan"
    model: "gpt-4"
    prompt: """
    You are a